# Gestión Moderna de Portafolio
### Autores Bernardo León y Carlos Zapata
### (C) Copyright 2023

## Capitulo 6: Medida Omega $\Omega$


In [118]:
%pip install --quiet yfinance

In [119]:
#Librerías usadas
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

### Ejemplo 6.1

In [133]:
# Información histórica para las acciones: AAPL, AMZN, GOOG y MSFT
fechai = "2015-12-01"
fechaf = "2020-12-31"
periodicidad = "1Mo"
activos = ["AAPL","AMZN","GOOGL","MSFT"]
precios = yf.download(activos,start=fechai,end=fechaf,interval=periodicidad)['Adj Close'].dropna()
retornos = np.log(precios/precios.shift(1)).dropna()
mu = retornos.mean()
cov = retornos.cov()
var = np.diag(cov)
sigma = np.sqrt(var)
n = len(mu)

[*********************100%***********************]  4 of 4 completed


In [134]:
# Calculo excesos y umbral h
h = 0
excesos = retornos-h
pos_ret = excesos[excesos>h].sum()
neg_ret = excesos[excesos<h].sum()
omegai = pos_ret / (-neg_ret )
omegai

AAPL     2.217804
AMZN     2.314771
GOOGL    1.734004
MSFT     3.257868
dtype: float64

# Optimización usando CVXPY

In [ ]:
def omega(retornos, h):
    s = retornos.shape[0]
    n = retornos.shape[1]
    y = cp.Variable(n)
    u = cp.Variable(s)
    z = cp.Variable()
    constraints = [cp.matmul(retornos, y) + u >= h,
                   cp.sum(y) == 1,
                   cp.sum(u) == z,
                   cp.matmul(mu, y) >= h*z,
                   z >= 1e-05]
    objective = cp.Minimize(cp.matmul(mu, y) - h*z)
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.GLPK_MI)
    return y.value